<a href="https://colab.research.google.com/github/firmansyahr/Production-Performance-Analysis-and-Continuous-Improvement/blob/main/Notebook/01_data_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01 – Data Validation & Readiness Check

## Business Context
This notebook validates raw production, downtime, OEE, and SPC datasets
used for production performance analysis and continuous improvement.

## Objectives
- Ensure data completeness and consistency
- Validate manufacturing business rules
- Detect logical and temporal anomalies
- Prepare clean datasets for downstream analysis




In [29]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)


In [30]:
BASE_URL = "https://raw.githubusercontent.com/firmansyahr/Production-Performance-Analysis-and-Continuous-Improvement/"
BRANCH = "master"
RAW_PATH = f"{BASE_URL}{BRANCH}/data/raw/"

print("Trying to load data from:")
print(RAW_PATH)


Trying to load data from:
https://raw.githubusercontent.com/firmansyahr/Production-Performance-Analysis-and-Continuous-Improvement/master/data/raw/


In [31]:
df_minutely = pd.read_csv(
    RAW_PATH + "factory_data.csv",
    parse_dates=["timestamp"]
)

df_oee_day = pd.read_csv(
    RAW_PATH + "oee_by_day.csv"
)

df_oee_shift = pd.read_csv(
    RAW_PATH + "oee_by_shift.csv"
)

df_downtime = pd.read_csv(
    RAW_PATH + "downtime_pareto.csv"
)

df_spc = pd.read_csv(
    RAW_PATH + "spc_xbar_r.csv"
)


In [32]:
print("Minutely data :", df_minutely.shape)
print("OEE by day   :", df_oee_day.shape)
print("OEE by shift :", df_oee_shift.shape)
print("Downtime     :", df_downtime.shape)
print("SPC          :", df_spc.shape)


Minutely data : (8640, 14)
OEE by day   : (8, 10)
OEE by shift : (20, 11)
Downtime     : (23, 4)
SPC          : (144, 6)


In [33]:
df_minutely.info()
df_minutely.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8640 entries, 0 to 8639
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   timestamp         8640 non-null   datetime64[ns]
 1   machine           8640 non-null   object        
 2   is_running        8640 non-null   int64         
 3   cause_Mechanical  8640 non-null   int64         
 4   cause_Electrical  8640 non-null   int64         
 5   cause_Changeover  8640 non-null   int64         
 6   cause_Blocked     8640 non-null   int64         
 7   cause_Starved     8640 non-null   int64         
 8   cause_Quality     8640 non-null   int64         
 9   units             8640 non-null   int64         
 10  scrap             8640 non-null   int64         
 11  good_units        8640 non-null   int64         
 12  shift             8640 non-null   object        
 13  day               8640 non-null   object        
dtypes: datetime64[ns](1), in

,timestamp,machine,is_running,cause_Mechanical,cause_Electrical,cause_Changeover,cause_Blocked,cause_Starved,cause_Quality,units,scrap,good_units,shift,day
0,2025-01-01 06:00:00,M1,1,0,0,0,0,0,0,7,0,7,A,2025-01-01
1,2025-01-01 06:00:00,M2,1,0,0,0,0,0,0,6,0,6,A,2025-01-01
2,2025-01-01 06:01:00,M1,1,0,0,0,0,0,0,10,0,10,A,2025-01-01
3,2025-01-01 06:01:00,M2,1,0,0,0,0,0,0,7,0,7,A,2025-01-01
4,2025-01-01 06:02:00,M1,1,0,0,0,0,0,0,6,0,6,A,2025-01-01


In [34]:
df_minutely.isna().sum()


,0
timestamp,0
machine,0
is_running,0
cause_Mechanical,0
cause_Electrical,0
cause_Changeover,0
cause_Blocked,0
cause_Starved,0
cause_Quality,0
units,0


Expected:
- No missing values in timestamp, machine, is_running


In [35]:
#Rule 1 — good_units = units − scrap
invalid_good_units = df_minutely[
    df_minutely["good_units"] != df_minutely["units"] - df_minutely["scrap"]
]

invalid_good_units.shape

#Rule 2 — If machine is down, no production
invalid_units_when_down = df_minutely[
    (df_minutely["is_running"] == 0) &
    (df_minutely["units"] > 0)
]

invalid_units_when_down.shape

#Rule 3 — Scrap cannot exceed units
(df_minutely["scrap"] > df_minutely["units"]).sum()

np.int64(0)

In [36]:
#Time & Shift Validation

df_minutely["hour"] = df_minutely["timestamp"].dt.hour
def validate_shift(row):
    h = row["hour"]
    if 6 <= h < 14:
        return row["shift"] == "A"
    elif 14 <= h < 22:
        return row["shift"] == "B"
    else:
        return row["shift"] == "C"

df_minutely["shift_valid"] = df_minutely.apply(validate_shift, axis=1)
df_minutely["shift_valid"].value_counts()


,count
shift_valid,
True,8640


In [37]:
#Daily Aggregation Sanity Check

daily_summary = (
    df_minutely
    .groupby(["day", "machine"])
    .agg(
        planned_min=("timestamp", "count"),
        running_min=("is_running", "sum"),
        total_units=("units", "sum"),
        good_units=("good_units", "sum"),
        scrap=("scrap", "sum")
    )
    .reset_index()
)

daily_summary.head()


,day,machine,planned_min,running_min,total_units,good_units,scrap
0,2025-01-01,M1,1080,975,5874,5851,23
1,2025-01-01,M2,1080,1046,6354,6336,18
2,2025-01-02,M1,1440,1285,7861,7837,24
3,2025-01-02,M2,1440,1396,8348,8314,34
4,2025-01-03,M1,1440,1201,7207,7185,22


In [38]:
#Cross-check vs OEE by Day

# Merge daily summary (calculated) with OEE reference data
cross_check = daily_summary.merge(
    df_oee_day,
    on=["day", "machine"],
    how="left",
    suffixes=("_calc", "_oee")
)

# Display cross-check columns safely
cross_check[[
    "day",
    "machine",
    "planned_min_calc",
    "running_min_calc",
    "total_units_calc",
    "good_units_calc",
    "planned_min_oee",
    "running_min_oee",
    "total_units_oee",
    "good_units_oee",
    "availability",
    "performance",
    "quality",
    "oee"
]].head()


,day,machine,planned_min_calc,running_min_calc,total_units_calc,good_units_calc,planned_min_oee,running_min_oee,total_units_oee,good_units_oee,availability,performance,quality,oee
0,2025-01-01,M1,1080,975,5874,5851,1080,975,5874,5851,0.902778,1.004103,0.996084,0.902932
1,2025-01-01,M2,1080,1046,6354,6336,1080,1046,6354,6336,0.968519,1.012428,0.997167,0.977778
2,2025-01-02,M1,1440,1285,7861,7837,1440,1285,7861,7837,0.892361,1.019585,0.996947,0.907060
3,2025-01-02,M2,1440,1396,8348,8314,1440,1396,8348,8314,0.969444,0.996657,0.995927,0.962269
4,2025-01-03,M1,1440,1201,7207,7185,1440,1201,7207,7185,0.834028,1.000139,0.996947,0.831597


In [39]:
#SPC Dataset Sanity Check
df_spc.describe()

,count,xbar,s,R
count,144.000000,144.000000,144.000000,144.000000
mean,54.222222,6.036390,2.446937,11.069444
std,11.011402,0.389644,0.276762,1.905651
min,13.000000,4.983333,1.553797,6.000000
25%,53.750000,5.783333,2.297734,10.000000
50%,60.000000,6.050000,2.439060,11.000000
75%,60.000000,6.266667,2.601241,12.000000
max,60.000000,7.095238,3.101804,17.000000


Expected:
- No negative values for xbar, R, or count
- Variability metrics within reasonable range

## Key Findings
- No critical missing values detected
- Logical relationships between units, scrap, and good_units are consistent
- Shift assignment aligns with timestamp rules
- Aggregated production data aligns with OEE reference tables
- Dataset is validated and ready for OEE, RCA, and SPC analysis
